In [0]:
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
import torch

In [2]:
# GPU設定を確認
# ランタイム＞ランタイムのタイプを変更でGPUのチェックボックスをONにして実行する
import torch
torch.cuda.is_available()

True

In [3]:
# Google driveに接続
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
cd /content/gdrive/My Drive/Colab Notebooks/pytorch_voc_ssd

/content/gdrive/My Drive/Colab Notebooks/pytorch_voc_ssd


In [5]:
ls

box_utils.py  layers/       pytorch_voc_ssd.ipynb  train.py  weights/
data/         __pycache__/  ssd.py                 utils/


In [0]:
from data import *
from utils.augmentations import SSDAugmentation
from layers.modules import MultiBoxLoss
from ssd import build_ssd
import os
import sys
import time
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.init as init
import torch.utils.data as data
import numpy as np
import argparse

In [0]:
def str2bool(v):
    return v.lower() in ("yes", "true", "t", "1")

In [0]:
args = {'dataset':'VOC',
        'dataset_root':VOC_ROOT,
        'basenet':'vgg16_reducedfc.pth',
        'batch_size':8,
        'resume':'',
        'start_iter':0,
        'num_workers':4,
        'cuda':True,
        'lr':1e-3,
        'momentum':0.9,
        'weight_decay':5e-4,
        'gamma':0.1,
        'visdom':False,
        'save_folder':'weights/'
       }

In [0]:
if torch.cuda.is_available():
    if args['cuda']:
        torch.set_default_tensor_type('torch.cuda.FloatTensor')
    if not args['cuda']:
        print("WARNING: It looks like you have a CUDA device, but aren't " +
              "using CUDA.\nRun with --cuda for optimal training speed.")
        torch.set_default_tensor_type('torch.FloatTensor')
else:
    torch.set_default_tensor_type('torch.FloatTensor')

In [0]:
if not os.path.exists(args['save_folder']):
    os.mkdir(args['save_folder'])

In [0]:
cfg = voc
dataset = VOCDetection(root=args['dataset_root'],
                       transform=SSDAugmentation(cfg['min_dim'],
                                                 MEANS))

In [0]:
ssd_net = build_ssd('train', cfg['min_dim'], cfg['num_classes'])
net = ssd_net

In [0]:
if args['cuda']:
    net = torch.nn.DataParallel(ssd_net)
    cudnn.benchmark = True

In [14]:
if args['resume']:
    print('Resuming training, loading {}...'.format(args['resume']))
    ssd_net.load_weights(args['resume'])
else:
    vgg_weights = torch.load(args['save_folder'] + args['basenet'])
    print('Loading base network...')
    ssd_net.vgg.load_state_dict(vgg_weights)

Loading base network...


In [0]:
if args['cuda']:
    net = net.cuda()

In [16]:
def xavier(param):
    init.xavier_uniform(param)

def weights_init(m):
    if isinstance(m, nn.Conv2d):
        xavier(m.weight.data)
        m.bias.data.zero_()

if not args['resume']:
    print('Initializing weights...')
    # initialize newly added layers' weights with xavier method
    ssd_net.extras.apply(weights_init)
    ssd_net.loc.apply(weights_init)
    ssd_net.conf.apply(weights_init)

Initializing weights...


In [0]:
optimizer = optim.SGD(net.parameters(), lr=args['lr'], momentum=args['momentum'],
                      weight_decay=args['weight_decay'])
criterion = MultiBoxLoss(cfg['num_classes'], 0.5, True, 0, True, 3, 0.5,
                         False, args['cuda'])

In [18]:
net.train()
# loss counters
loc_loss = 0
conf_loss = 0
epoch = 0
print('Loading the dataset...')

epoch_size = len(dataset) // args['batch_size']
print('Training SSD on:', dataset.name)
print('Using the specified args:')
print(args)

step_index = 0

Loading the dataset...
Training SSD on: VOC0712
Using the specified args:
{'dataset': 'VOC', 'dataset_root': '/content/gdrive/My Drive/Colab Notebooks/pytorch_voc_ssd/data/VOCdevkit', 'basenet': 'vgg16_reducedfc.pth', 'batch_size': 16, 'resume': '', 'start_iter': 0, 'num_workers': 4, 'cuda': True, 'lr': 0.001, 'momentum': 0.9, 'weight_decay': 0.0005, 'gamma': 0.1, 'visdom': False, 'save_folder': 'weights/'}


In [0]:
data_loader = data.DataLoader(dataset, args['batch_size'],
                              num_workers=args['num_workers'],
                              shuffle=True, collate_fn=detection_collate,
                              pin_memory=True)

In [0]:
def adjust_learning_rate(optimizer, gamma, step):
    """Sets the learning rate to the initial LR decayed by 10 at every
        specified step
    # Adapted from PyTorch Imagenet example:
    # https://github.com/pytorch/examples/blob/master/imagenet/main.py
    """
    lr = args['lr'] * (gamma ** (step))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


def xavier(param):
    init.xavier_uniform(param)


def weights_init(m):
    if isinstance(m, nn.Conv2d):
        xavier(m.weight.data)
        m.bias.data.zero_()

In [0]:
batch_iterator = None
for iteration in range(args['start_iter'], cfg['max_iter']):
    if (not batch_iterator) or (iteration % epoch_size ==0):
        batch_iterator = iter(data_loader)
        loc_loss = 0
        conf_loss = 0
        epoch += 1

# mod

    if iteration in cfg['lr_steps']:
        step_index += 1
        adjust_learning_rate(optimizer, args['gamma'], step_index)

    # load train data
    images, targets = next(batch_iterator)

    if args['cuda']:
        images = Variable(images.cuda())
        targets = [Variable(ann.cuda(), volatile=True) for ann in targets]
    else:
        images = Variable(images)
        targets = [Variable(ann, volatile=True) for ann in targets]
    # forward
    t0 = time.time()
    out = net(images)
    # backprop
    optimizer.zero_grad()
    loss_l, loss_c = criterion(out, targets)
    loss = loss_l + loss_c
    loss.backward()
    optimizer.step()
    t1 = time.time()
    loc_loss += loss_l.data[0]
    conf_loss += loss_c.data[0]

    if iteration % 10 == 0:
        print('timer: %.4f sec.' % (t1 - t0))
        print('iter ' + repr(iteration) + ' || Loss: %.4f ||' % (loss.data[0]), end=' ')

#    if args.visdom:
#        update_vis_plot(iteration, loss_l.data[0], loss_c.data[0],
#                        iter_plot, epoch_plot, 'append')

#    if iteration != 0 and iteration % 5000 == 0:
#        print('Saving state, iter:', iteration)
#        torch.save(ssd_net.state_dict(), 'weights/ssd300_COCO_' +
#                   repr(iteration) + '.pth')
torch.save(ssd_net.state_dict(),
           args['save_folder'] + '' + args['dataset'] + '.pth')
